In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display
import pandas as pd
import spot
import sys
spot.setup(show_default='.a')
pd.options.display.float_format = '{: .0f}'.format
pd.options.display.latex.multicolumn_format = 'c'

In [25]:
rerun = True

In [3]:
%%bash
ltl3ba -v
ltl3tela -v
ltl2tgba --version
delag --version
ltl2dgra --version # Rabinizer 4

LTL3BA 1.1.3
LTL3TELA 1.3.0 (using Spot 2.7.4)
ltl2tgba (spot) 2.7.4

Copyright (C) 2019  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Name: owl
Version: 18.06
Name: owl
Version: 18.06


In [4]:
def generate(n=1000,func=(lambda x: True),filename=None,priorities='',ap=['a','b','c','d','e']):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,
                     ltl_priorities=priorities,
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

In [5]:
f_rand = 'formulae/atva19/rand.ltl'
f_patterns = 'formulae/atva19/patterns.ltl'
# generate(1000, filename = f_rand)

### Deterministic automata

In [6]:
d_tools = {
    "ltl3tela-D1": "ltl3tela -D1 -f %f > %O",
    "ltl2tgba-DG": "ltl2tgba -DG %f > %O",
    "delag": "delag %f > %O",
    "rabinizer4": "ltl2dgra %f > %O"
}
d_order = ["ltl3tela-D1", "ltl2tgba-DG", "delag", "rabinizer4"]
d_cols = ["states", "edges", "transitions", "acc"]

In [7]:
d_csv_rand = 'formulae/atva19/det.rand.csv'
d_data_rand = LtlcrossRunner(d_tools, formula_files = [f_rand], res_filename = d_csv_rand, cols = d_cols)
if not rerun:
    d_data_rand.run_ltlcross(automata = False, timeout = '60')
d_data_rand.parse_results()

In [8]:
d_data_rand.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]

column,states,edges,transitions,acc
tool,,,,
ltl3tela-D1,5934,18520,54210,1268
ltl2tgba-DG,6799,24131,63428,1575
delag,7176,71672,71683,3089
rabinizer4,7581,31099,72592,2786


In [9]:
d_csv_patterns = 'formulae/atva19/det.patterns.csv'
d_data_patterns = LtlcrossRunner(d_tools, formula_files = [f_patterns], res_filename = d_csv_patterns, cols = d_cols)
if not rerun:
    d_data_patterns.run_ltlcross(automata = False, timeout = '60')
d_data_patterns.parse_results()

In [10]:
d_data_patterns.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]

column,states,edges,transitions,acc
tool,,,,
ltl3tela-D1,2536,10641,134591,454
ltl2tgba-DG,3905,26643,155470,652
delag,8661,2209807,2209841,1196
rabinizer4,2969,12358,137907,1133


In [11]:
#to = d_data_patterns.exit_status
#to[to == "timeout"].count()
#len(d_data_patterns.values.dropna().index)

In [27]:
d_data_patterns.smaller_than('ltl3tela-D1', 'ltl2tgba-DG')

column                                                          states  \
tool                                                       ltl2tgba-DG   
form_id formula                                                          
50      G!p0 | F(p0 & (!p1 W p2))                                    5   
68      G((p0 & XFp1) -> XF(p1 & Fp2))                              10   
71      G((p0 & Fp1) -> (((p2 & X(!p1 U p3)) -> X(!p1 U...          19   
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...          16   
73      G(p0 -> F(p1 & XFp2))                                        6   
77      G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4))...          11   
78      G(p0 -> F(p1 & !p2 & X(!p2 U p3)))                           7   
94      G(p0 -> (p1 U (Gp2 | Gp3)))                                  8   
172     G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2...          81   
180     G((!p0 & Xp0) -> X(!(!p0 & Xp0) U (!p1 & Xp1)))              9   
310     Fp0 U Gp1                                                    3   
325     Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1))                   6   
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))                5   
332     GFa1 U G(GFa0 U Xb)                                         21   
333     GFa1 U G(GFa0 U XXb)                                        22   
334     GFa1 U G(GFa0 U XXXb)                                       23   
335     GFa1 U G(GFa0 U XXXXb)                                      24   
336     GFa1 U G(GFa0 U XXXXXb)                                     25   
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                              257   
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                             258   
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                            259   
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                           260   
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                          261   
387     G(p1 -> (p1 U (p2 & (p2 U (p3 & (p3 U p4))))))              12   
388     G(p1 -> (p1 U (p2 & (p2 U (p3 & (p3 U (p4 & (p4...          51   

column                                                                  
tool                                                       ltl3tela-D1  
form_id formula                                                         
50      G!p0 | F(p0 & (!p1 W p2))                                    4  
68      G((p0 & XFp1) -> XF(p1 & Fp2))                               4  
71      G((p0 & Fp1) -> (((p2 & X(!p1 U p3)) -> X(!p1 U...           8  
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...           6  
73      G(p0 -> F(p1 & XFp2))                                        4  
77      G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4))...          10  
78      G(p0 -> F(p1 & !p2 & X(!p2 U p3)))                           4  
94      G(p0 -> (p1 U (Gp2 | Gp3)))                                  7  
172     G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2...          34  
180     G((!p0 & Xp0) -> X(!(!p0 & Xp0) U (!p1 & Xp1)))              7  
310     Fp0 U Gp1                                                    2  
325     Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1))                   4  
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))                4  
332     GFa1 U G(GFa0 U Xb)                                         11  
333     GFa1 U G(GFa0 U XXb)                                        12  
334     GFa1 U G(GFa0 U XXXb)                                       13  
335     GFa1 U G(GFa0 U XXXXb)                                      14  
336     GFa1 U G(GFa0 U XXXXXb)                                     15  
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                               11  
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                              12  
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                             13  
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                            14  
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                           15  
387     G(p1 -> (p1 U (p2 & (

In [26]:
d_data_patterns.smaller_than('ltl2tgba-DG', 'ltl3tela-D1')

Empty DataFrame
Columns: [(states, ltl2tgba-DG), (states, ltl3tela-D1)]
Index: []

### Nondeterministic automata

In [13]:
import os
os.environ['SPOT_HOA_TOLERANT']='TRUE'

In [14]:
n_tools = {
    "ltl3tela": "ltl3tela -f %f > %O",
    "ltl2tgba": "ltl2tgba %f > %O",
    "ltl2tgba-G": "ltl2tgba -G %f > %O",
    "ltl3ba": "ltldo 'ltl3ba -H2' -f %f > %O",
}
n_order = ["ltl3tela", "ltl2tgba", "ltl2tgba-G", "ltl3ba"]
n_cols = ["states", "edges", "transitions", "acc"]

In [15]:
n_csv_rand = 'formulae/atva19/nondet.rand.csv'
n_data_rand = LtlcrossRunner(n_tools, formula_files = [f_rand], res_filename = n_csv_rand, cols = n_cols)
if not rerun:
    n_data_rand.run_ltlcross(automata = False, timeout = '60')
n_data_rand.parse_results()

In [16]:
n_data_rand.cummulative(col = n_cols).unstack(level = 0).loc[n_order, n_cols]

column,states,edges,transitions,acc
tool,,,,
ltl3tela,5109,12481,41390,1135
ltl2tgba,5413,13059,43374,1034
ltl2tgba-G,5391,13144,43457,1041
ltl3ba,6103,15636,51078,1616


In [17]:
n_csv_patterns = 'formulae/atva19/nondet.patterns.csv'

In [18]:
n_data_patterns = LtlcrossRunner(n_tools, formula_files = [f_patterns], res_filename = n_csv_patterns, cols = n_cols)
if not rerun:
    n_data_patterns.run_ltlcross(automata = False, timeout = '60')
n_data_patterns.parse_results()

In [19]:
n_data_patterns.cummulative(col = n_cols).unstack(level = 0).loc[n_order, n_cols]

column,states,edges,transitions,acc
tool,,,,
ltl3tela,2378,20718,118535,544
ltl2tgba,2651,8721,224648,502
ltl2tgba-G,2398,20555,118469,642
ltl3ba,4654,21180,995971,822


In [20]:
n_data_patterns.smaller_than('ltl3tela', 'ltl2tgba-G')

column                                                         states         
tool                                                       ltl2tgba-G ltl3tela
form_id formula                                                               
50      G!p0 | F(p0 & (!p1 W p2))                                   5        4
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...         10        8
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))               6        4
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                               9        8
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                             10        9
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                            11       10
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                           12       11
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                          13       12
342     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U Xb)))                    11        9
343     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXb)))                   12       10
344     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXb)))                  13       11
345     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXXb)))                 14       12
346     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXXXb)))                15       13

In [21]:
n_data_patterns.smaller_than('ltl2tgba-G', 'ltl3tela')

Empty DataFrame
Columns: [(states, ltl2tgba-G), (states, ltl3tela)]
Index: []

In [22]:
#rerun = True
#test_csv = 'formulae/atva19/test.csv'
#test_tools = {
#    'split-obligation': 'ltl3tela -D1 -f %f > %O',
#    'whole-obligation': 'ltl3tela -D1 -y1 -f %f > %O'
#}
#test_data = LtlcrossRunner(test_tools, formula_files = [f_patterns], res_filename = test_csv,
#                                 cols = ['states'])
#if not rerun:
#    test_data.run_ltlcross(timeout = '30')
#test_data.parse_results(test_csv)

In [23]:
#test_cols = ['states'];
#test_data.cummulative(col = test_cols).unstack(level = 0).loc[['split-obligation', 'whole-obligation'], test_cols]

In [24]:
#test_data.smaller_than('split-obligation', 'whole-obligation')